<img src="images/header.png">

# Алгоритмы интеллектуальной обработки больших объемов данных

## Лекция 3. K-means и EM алгоритм

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Сегодня на лекции

* Смесь нормальных распределений и EM
* K-means и его модификации

## На предыдущей лекции

**Дано.** Признаковые описания $N$ объектов $\mathbf{x} = (x_1, \ldots, x_m) \in \mathcal{X}$, образующие тренировочный набор данных $X$  

**Найти.** Модель из семейства параметрических функций 

$$H = \{h(\mathbf{x, \mathbf{\theta}}): \mathcal{X} \times \Theta \rightarrow \mathcal{Y} \mid \mathcal{Y} = \{1, \ldots, K\}\},$$
ставящую в соответствие произвольному $\mathbf{x} \in \mathcal{X}$ один из $K$ кластеров так, чтобы объекты внутри одного кластера были похожи, а объекты из разных кластеров различались  

**Алгоритмы.** Hierarchical Clustering, dbscan


### Нужно отметиться на лекции

https://sphere.mail.ru/

# Смесь нормальных распределений и EM
<img src="images/local.png">

## Гаусс, Карл Фридрих (1777-1855)

<img src="images/gauss.jpg">

* Не открыл распределение Гаусса
* Открыл все остальное

Иога́нн Карл Фри́дрих Га́усс (нем. Johann Carl Friedrich Gauß; 30 апреля 1777, Брауншвейг — 23 февраля 1855, Гёттинген) — немецкий математик, механик, физик, астроном и геодезист[12]. Считается одним из величайших математиков всех времён, «королём математиков»[13]

https://ru.wikipedia.org/wiki/%D0%93%D0%B0%D1%83%D1%81%D1%81,_%D0%9A%D0%B0%D1%80%D0%BB_%D0%A4%D1%80%D0%B8%D0%B4%D1%80%D0%B8%D1%85

## Начнем с простого (смоделируем один кластер)

**Данные**   
Координаты точек попаданий по мишени из гауссовской пушки  

**Задача**  
Определить, куда смещен прицел  

<img src="images/target.jpg">

## Многомерное нормальное распределение

$$\mathcal{N(\mathbf{x} | \mathbf{\mu}, \mathbf{\Sigma}}) = \frac{1}{(2 \pi)^{D/2}} \frac{1}{|\mathbf{\Sigma}|^{1/2}} \exp \left\{-\frac{1}{2}(\mathbf{x} - \mathbf{\mu})^T \mathbf{\Sigma^{-1}} (\mathbf{x} - \mathbf{\mu})\right\}$$

**Параметры**  


${D}$-мерный вектор средних

$$\mathbf{\mu}$$

$D \times D$-мерная матрица ковариации   


$$\mathbf{\Sigma} = E[(\mathbf{x} - \mathbf{\mu})(\mathbf{x} - \mathbf{\mu})^T]$$



Матрица ковариации симметрична

$\mathbf{x}$ - размерности $D$  


$$D = 2$$
<img src="images/multi.png">

<img src="images/gnormal.png">

$$\mathbf{\Sigma} = \text{diag}(\sigma_i)$$
<img src="images/dnormal.png">

$$\mathbf{\Sigma} = \sigma I$$
<img src="images/enormal.png">

Чтобы выяснить параметры распределения модели необходимо вычислить параметры:  
$\mu$: $D$ параметров
$\Sigma$: $(D+1)*D/2$, так как матрица симметричная, считается из арифметической прогрессии.

Итого: $(D+3)*D/2$ параметров 

Это довольно много, поэтому делаются предположения о матрице ковариации

## Формализуем задачу

Имеется набор данных
$$X = \{\mathbf{x}_n \in R^2\}$$

Предположение
$$p(\mathbf{x}_n) \sim \mathcal{N(\mathbf{x} | \mathbf{\mu}, \mathbf{\Sigma}}), \quad \mu \in R^2, \;\; \mathbf{\Sigma} \in R^{2 \times 2}$$

Требуется найти вектор средних $\mu$ и матрицу ковариации $\mathbf{\Sigma}$

## Maximum likelihood

Принцип максимального правдоподобия  

Пусть дано семейство параметрических моделей $h(\mathbf{x}, \mathbf{\theta})$. Выбираем вектор параметров $\theta$, максимизирующий функцию правдоподобия (likelihood) $p(\mathcal{D} | \theta)$, соответствующую рассматриваемому семейству моделей.

Правдоподобие  
$$L(X | \mu, \mathbf{\Sigma}) = \prod_{n=1}^N \mathcal{N}(\mathbf{x_n} | \mathbf{\mu}, \mathbf{\Sigma}) \rightarrow \max_{\mu, \mathbf{\Sigma}}$$

Решение
$$\mu_{ML} = \frac 1 N \sum_{n=1}^N \mathbf{x_n}, \quad \mathbf{\Sigma}_{ML} = \frac 1 N \sum_{n=1}^N (\mathbf{x_n} - \mu_{ML}) (\mathbf{x_n} - \mu_{ML})^T$$

## Old Faithful data set

$D$ = date of recordings in month (in August)  
$X$ = duration of the current eruption in minutes   
$Y$ = waiting time until the next eruption in minutes  



<img src="images/of1.jpg">

<img src="images/of2.png">

## Смесь нормальных распределений

*Скрытая* $K$-мерная переменная $\mathbf{z}$ - принадлежность объекта к одному из кластеров

$$p(z_k = 1) = \pi_k, \quad z_k \in \{0, 1\}, \quad \sum_k z_k = 1 \quad\rightarrow\quad p(\mathbf{z}) = \prod_k \pi_k^{z_k}$$

Распределение $\mathbf{x}$ для каждого из $K$ кластеров

$$p(\mathbf{x} | \mathbf{z_k}) = \mathcal{N}(\mathbf{x} | \mathbf{\mu}_k, \mathbf{\Sigma}_k) \quad \rightarrow \quad p(\mathbf{x} | \mathbf{z}) = \prod_k \mathcal{N}(\mathbf{x} | \mathbf{\mu}_k, \mathbf{\Sigma}_k)^{z_k}$$


Смесь нормальных распределений
$$
p(\mathbf{x}) = \sum_k \pi_k \mathcal{N}(\mathbf{x} | \mathbf{\mu}_k, \mathbf{\Sigma}_k)
$$


Введем $z$ - скрытая $k$-мерная переменная 
Каждый $k$-й компонент $z$ отражает принадлежит точка кластеру или нет
$k$ - количество кластеров

Введем на $z_k$ априорное распределение. Т.е. допустим, что ничего не знаем про нашу задачу, но хотим иметь хоть что-то.
Нужно прийти к векторной форме. Вероятность $\mathbf{z}$ в точности будет равна $\pi_k$

Каким может быть $x$ если он принадлежит $k$-му кластеру?
Приведем это в векторную форму. Делаем такой же трюк.

То, что видим на картинке это $p(x|\pi_k, \mu, \Sigma)$, далее будем опускать параметры

$p(x) = p(x|z)p(z)$, хотим избавиться от $z$, поэтому просуммируем по $z_k$

введем еще величину $\gamma(z_k)$ - вероятность $z_k = 1$ после того, как пронаблюдали $x$

формула Байеса

Апостериорная вероятность принадлежности к $k$ кластеру (априорная равна $\pi_k$)
$$
\gamma(z_k) = p(z_k = 1 | \mathbf{x}) = \frac{p(z_k=1) p(\mathbf{x} | z_k = 1)}{\sum_{j=1}^K p(z_j=1) p(\mathbf{x} | z_j = 1)} = \frac{\pi_k \mathcal{N}(\mathbf{x} | \mathbf{\mu}_k, \mathbf{\Sigma}_k)}{\sum_{j=1}^K \pi_j \mathcal{N}(\mathbf{x} | \mathbf{\mu}_j, \mathbf{\Sigma}_j)}
$$


## Maximum Likelihood
Функция правдоподобия
$$
\log p(\mathbf{X} | \mathbf{\pi}, \mathbf{\mu}, \mathbf{\Sigma}) = \sum_{n=1}^N \log \sum_k \pi_k \mathcal{N}(\mathbf{x}_n | \mathbf{\mu}_k, \mathbf{\Sigma}_k) \rightarrow \max_{\mathbf{\pi}, \mathbf{\mu}, \mathbf{\Sigma}}
$$

Сложности
* схлопывание компонент
* переименование кластеров
* невозможно оптимизировать аналитически



Вероятности очень малы. Переходят к логарифму. Функция непрерывно возрастающая, поэтому максимум функции и максимум логарифма функции будут совпадать.

Наблюдаем N объектов.

Возьмум функцию правдоподобия, продифференцируем по всем параметрам, приравняем к нулю.

Схлопывание компонент - реакция на выбросы. Нужно следить за этой ситуацией. Сигма стремится к нулю    
Переименование - дважды запустив алгоритм можем получить такой же по сути, но отличающийся по меткам результат  
В одномерном случае оптимизировать аналитически возможно  

Есть набор моделей. Хотим выбрать ту, которая наилучшим образом сообветствует нашим данным.

Дифференцируем функцию правдоподобия
$$
N_k = \sum_{n=1}^N \gamma(z_{nk}), \;\; \mu_k = \frac 1 {N_k} \sum_{n=1}^N \gamma(z_{nk}) \mathbf{x}_n
$$
$$
\Sigma_k = \frac 1 {N_k} \sum_{n=1}^N \gamma(z_{nk}) (\mathbf{x}_n - \mu_k)^T (\mathbf{x}_n - \mu_k)
$$
$$
\pi_k = \frac{N_k}{N}
$$


## Expectation Maximization

**E** Expectation: при фиксированных $\mu_k, \Sigma_k, \pi_k$
$$
\gamma(z_{nk}) = \frac{\pi_k \mathcal{N} (\mathbf{x}_n | \mu_k, \Sigma_k)}{\sum_{j=1}^K \pi_j \mathcal{N} (\mathbf{x}_n | \mu_j, \Sigma_j)}
$$
**M** Maximization: при фиксированных $\gamma(z_{nk})$
$$
N_k = \sum_{n=1}^N \gamma(z_{nk}), \;\; \mu_k = \frac 1 {N_k} \sum_{n=1}^N \gamma(z_{nk}) \mathbf{x}_n
$$
$$
\Sigma_k = \frac 1 {N_k} \sum_{n=1}^N \gamma(z_{nk}) (\mathbf{x}_n - \mu_k)(\mathbf{x}_n - \mu_k)^T
$$
$$
\pi_k = \frac{N_k}{N}
$$
**S** Остановиться при достижении сходимости


<img src="images/gauss.png">

## EM-алгоритм

**Дано.**  
Известно распределение $P(\mathbf{X}, \mathbf{Z} | \theta)$, где $\mathbf{x}$ - наблюдаемые переменные, а $\mathbf{z}$ - скрытые. 

**Найти.**  
$\theta$,  максимизирующее $P(\mathbf{X} | \theta)$.

**E** вычислить $P(\mathbf{Z} | \mathbf{X}, \theta^{old})$ при фиксированном $\theta^{old}$
**M** вычислить $\theta^{new} = \arg \max_{\theta} \mathcal{Q} (\theta, \theta^{old})$, где
$$
\mathcal{Q} (\theta, \theta^{old}) = E_\mathbf{Z}[\ln p(\mathbf{X}, \mathbf{Z} | \theta)] = \sum_{\mathbf{Z}} p(\mathbf{Z} | \mathbf{X}, \theta^{old}) \ln p(\mathbf{X}, \mathbf{Z} | \theta))
$$

*Улучшение:* ввести априорное распределение $p(\theta)$


[Maximum Likelihood from Incomplete Data via the EM Algorithm](http://web.mit.edu/6.435/www/Dempster77.pdf)

## Сходимость

<img src="images/convergence.png">

[The Expectation Maximization Algorithm: A short tutorial](http://www.cs.cmu.edu/~dgovinda/pdf/recog/EM_algorithm-1.pdf)

## Различные смеси

<img src="images/mixtures.png">

## Перерыв
## Тест

## K-means и его модификации
<img src="images/k-cover.png">


## K-means

Пусть $\Sigma_k = \epsilon I$, тогда
$$
p(\mathbf{x} | \mu_k, \Sigma_k) = \frac{1}{\sqrt{2\pi\epsilon}}\exp(-\frac{1}{2\epsilon}\|\mathbf{x}-\mathbf{\mu_k}\|^2)
$$
Рассмотрим стремление $\epsilon \rightarrow 0$
$$
\gamma(z_{nk}) = \frac{\pi_k \exp(-\frac{1}{2\epsilon}\|\mathbf{x}_n-\mathbf{\mu_k}\|^2)}{\sum_j \pi_j \exp(-\frac{1}{2\epsilon}\|\mathbf{x}_n-\mathbf{\mu_j}\|^2)} \rightarrow r_{nk} = \begin{cases}
1, \; \text{для } k = \arg \min_j \|\mathbf{x}_n - \mathbf{\mu_j}\|^2 \\
0, \; \text{иначе}
\end{cases}
$$
Функция правдоподобия
$$
E_\mathbf{Z}[\ln p(\mathbf{X}, \mathbf{Z} | \mu, \Sigma, \pi)] \rightarrow -\sum_{n=1}^N \sum_{k=1}^K r_{nk} \| \mathbf{x}_n - \mu_k \|^2 + const
$$
Вектор средних
$$
\mathbf{\mu}_k = \frac{\sum_n r_{nk} \mathbf{x_n}}{\sum_n r_{nk}} 
$$


## K-means

```
function kmeans(X, K):
	initialize N # number of objects
	initialize Mu = (mu_1 ... mu_K) # random centroids
	do:
		# E step
		for k in 1..K:
			for x in 1..N:
				compute r_nk # Cluster assignment
		# M step
		for k in 1..K:
			recompute mu_k # Update centroids
	until Mu converged
	J = loss(X, Mu)
	return Mu, J
```
Сложность $O(NK)$   
Локальная оптимизация

<img src="images/kmeans.png">

## Модификации k-means



* На каждом шаге работаем с $b$ случайно выбранными объектами из каждого кластера (mini-batch k-means)
<img src="images/mbatch.png">

* Критерий качества (k-medoids)
$$
\tilde J = \sum_{n=1}^N \sum_{k=1}^K r_{nk} d(\mathbf{x}_n, \mu_k)
$$
$d$ - функция расстояния, $\mu_k$ - один из объектов в кластере

* k-means++ - *умная* инициализация. [k-means++: The Advantages of Careful Seeding](http://theory.stanford.edu/~sergei/papers/kMeansPP-soda.pdf)

* онлайн k-means
$$\mu_k^{new} = \mu_k^{old} + \eta_n (\mathbf{x}_n - \mu_k^{old})$$

## Задача на дом

Рассмотреть смесь из $D$-мерных распределений Бернулли. В такой смеси $\mathbf{x}$ -- $D$-мерный бинарный вектор, каждый компонент $x_i$ которого подчиняется распределению Бернулли с параметром $\mu_{ki}$ при заданном векторе $\mu_k$:
$$
p(\mathbf{x} | \mu_k) = \prod_{i=1}^D \mu_{ki}^{x_i} (1-\mu_{ki})^{(1-x_i)}
$$
Вероятность $k$-го вектора $\mu_k$ равна $\pi_k$. Выписать выражения для E и M шагов при использовании EM-алгоритма для нахождения неизвестных параметров $\mu_k$ и $\pi_k$.


http://ai.stanford.edu/~chuongdo/papers/em_tutorial.pdf  
https://people.duke.edu/~ccc14/sta-663/EMAlgorithm.html  
http://www.machinelearning.ru/wiki/images/1/16/S04_matrix_calculations.pdf  
http://stats.stackexchange.com/questions/27436/how-to-take-derivative-of-multivariate-normal-density  
http://www.math.uwaterloo.ca/~hwolkowi//matrixcookbook.pdf  


### Вопросы
### Пожалуйста, напишите отзыв о лекции